In [ ]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
import java.util.Properties
import java.sql.{Connection, DriverManager, ResultSet}
import scala.util.Try
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.{DataFrame, SparkSession, SQLContext}
import org.apache.spark.sql.SaveMode._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import org.json4s._
//import spark.sqlContext.implicits._
def sc = spark.sparkContext
//import spark.implicits._
val spark = {
  SparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
val formatter = DateTimeFormat.forPattern("yyyy-MM-dd")

In [ ]:
var dataframe = spark.read.
      option("header", true).
      option("delimiter", ",").
      option("inferSchema",true).
      csv("taza users.csv")
var df=dataframe.withColumn("MyDate",to_date(col("timestamp"))).where(col("MyDate").isNotNull).persist// users of 2019-11-15

var dataframe1 = spark.read.
      option("header", true).
      option("delimiter", ",").
      option("inferSchema",true).
      csv("active.csv")
var newdf=dataframe1.withColumn("Date",to_date(col("active_date"))).where(col("Date").isNotNull).persist//active date of users

In [ ]:
///@st device id-i sessioni qani editor done-a exel
var ed=df
.select(col("session_id"),col("device_id").as("ed_dev"))
.where(col("event_type")==="editor_done")
.groupBy("session_id","ed_dev")
.agg(count("session_id").as("c"))

//////en userner@ ovqer unen session vortex uxix 3 hat editor donen are
var ed_3=ed
.withColumnRenamed("ed_dev","ed_3_dev")
.where(col("c")===3)

////// en userner@ ovqer unen session vortex 3ic avel ed en are 
var ed_m3=ed
.withColumnRenamed("ed_dev","ed_m3_dev")
.where(col("c")>3)

///heto sranc himan vra usern@ bajanvumen en userneri ov 0 ed en are ovqer unen session vortex 3 ed en are  ovqer 3ic avle ev ovqer 1-2

In [ ]:
var mydf=df.filter(col("event_type")==="editor_open").select("device_id").distinct// en userner@ ovqer eo-en arel

///////////////////////////////////userner voronq ed chen arel
var not_ed=mydf.join(ed,col("ed_dev")===col("device_id"),"left_outer").where(col("c").isNull).select(col("device_id").as("id")).distinct

///userner iranc sessionoerov vor ed arelen
var help=mydf.join(ed,col("ed_dev")===col("device_id"),"left_outer").where(col("c").isNotNull)

///////////////////userner@ ovqer unen session vortex 3 ed en arel
var exact_3_ed=help.where(col("c")===3).select(col("device_id").as("id")).distinct

// // //en userner@ ovqer chunen session vortex uxix 3ed en are                         
var not_3_ed=help.join(exact_3_ed,col("device_id")===col("id"),"left_outer").where(col("id").isNull).drop("id")

///////////////////////////////////////// userner ovqer 3ic pakas ed arac session chunen
var not_less_3_ed=not_3_ed.where(col("c")>3).select(col("device_id").as("id")).distinct
///////////////////////////////////////////////userner ovqer 3ic avel ed arac session chunen
var not_more_3_ed=not_3_ed.join(not_less_3_ed,col("device_id")===col("id"),"left_outer").where(col("id").isNull).select(col("device_id").as("id")).distinct


In [ ]:
/// device_idner@ @st segmenti  -lit@ uxaki heto group anelu hamara
var _0_ed=not_ed.withColumn("c_ed_0",lit(0))
var _1_2_ed=not_more_3_ed.withColumn("c_ed_2",lit(2))
var _3_ed=exact_3_ed.withColumn("c_ed_3",lit(3))
var _m3_ed=not_less_3_ed.withColumn("c_ed_4",lit(4))

In [ ]:
/////////// count of all users
var users=df
.groupBy("MyDate")
.agg(countDistinct("device_id").as("all"))

////// count of all  eo users
var eo_users=df
.filter(col("event_type")==="editor_open")
.groupBy(col("MyDate").as("eo_date"))
.agg(countDistinct("device_id").as("eo"))

/////////////////////// count of all ed users
var ed_users=df
.filter(col("event_type")==="editor_done")
.groupBy(col("MyDate").as("ed_date"))
.agg(countDistinct("device_id").as("ed"))

/////////////couunt of segemnt users
var _0_ed=not_ed
.withColumn("_0_ed_date",lit("2019-11-15"))
.groupBy("_0_ed_date")
.agg(count("id").as("_0_ed"))

var _1_2_ed=not_more_3_ed
.withColumn("_1_2_ed_date",lit("2019-11-15"))
.groupBy("_1_2_ed_date")
.agg(count("id").as("_1_2_ed"))
.persist///aranc persisti join aneluc cartesian errorer tali 
    
var _3_ed=exact_3_ed
.withColumn("_3_ed_date",lit("2019-11-15"))
.groupBy("_3_ed_date")
.agg(count("id").as("_3_ed"))

var _m3_ed=not_less_3_ed
.withColumn("_m3_ed_date",lit("2019-11-15"))
.groupBy("_m3_ed_date")
.agg(count("id").as("_m3_ed"))
.persist
//////////////////////////////////////8 hogu sxalem hashvum

In [ ]:
var all=users////////bolor@ prosto joina anum
.join(eo_users,users("MyDate")===eo_users("eo_date"),"left_outer")
.join(ed_users,users("MyDate")===ed_users("ed_date"),"left_outer")
.join(_0_ed,users("MyDate")===_0_ed("_0_ed_date"),"left_outer")
.join(_1_2_ed,users("MyDate")===_1_2_ed("_1_2_ed_date"),"left_outer")
.join(_3_ed,users("MyDate")===_3_ed("_3_ed_date"),"left_outer")
.join(_m3_ed,users("MyDate")===_m3_ed("_m3_ed_date"),"left_outer")
.drop("ed_date","_3_ed_date","_m3_ed_date","_0_ed_date","eo_date","_1_2_ed_date")

all.show ////////8 hogi korcnumem hashvarki mej

In [ ]:
////almost 1 day retention // chem hascre normal tesqi berel
var filter_newdf=newdf.where(col("active_date")==="2019-11-16").select(col("device_id"))

df///retention for all users
.select(col("device_id").as("all_id"))
.join(filter_newdf,col("all_id")===filter_newdf.col("device_id"),"left")
.where(col("device_id").isNotNull)
.distinct.count()

df ///retention for all eo users
.filter(col("event_type")==="editor_open")
.select(col("device_id").as("all_id"))
.join(filter_newdf,col("all_id")===filter_newdf.col("device_id"),"left")
.where(col("device_id").isNotNull)
.distinct.count()

df///retention for all  ed users
.filter(col("event_type")==="editor_done")
.select(col("device_id").as("all_id"))
.join(filter_newdf,col("all_id")===filter_newdf.col("device_id"),"left")
.where(col("device_id").isNotNull)
.distinct.count()

// ////almost retention by segment
 _m3_ed.join(filter_newdf,_m3_ed.col("id")===filter_newdf.col("device_id"),"left").where(col("device_id").isNotNull).count()
 _3_ed.join(filter_newdf,_3_ed.col("id")===filter_newdf.col("device_id"),"left").where(col("device_id").isNotNull).count()
_1_2_ed.join(filter_newdf,_1_2_ed.col("id")===filter_newdf.col("device_id"),"left").where(col("device_id").isNotNull).count()
_0_ed.join(filter_newdf,_0_ed.col("id")===filter_newdf.col("device_id"),"left").where(col("device_id").isNotNull).count()

In [ ]:
////////duration all users
import org.apache.spark.sql.types.{LongType}
var dur=df.select("session_id","timestamp","device_id")
.withColumn("ses_start",min(col("timestamp")).over(Window.partitionBy(col("session_id"))))
.withColumn("ses_end",max(col("timestamp")).over(Window.partitionBy(col("session_id"))))
.withColumn("dur",col("ses_end").cast(LongType)-col("ses_start").cast(LongType))
.drop("ses_start","ses_end","timestamp")
.distinct()//xi distinct???
.groupBy("device_id")
.agg(avg(col("dur")).as("avg_dur"))
.orderBy(col("avg_dur").desc)
.persist

////////////frequency all users
var freq=df.select("device_id","session_id")
.groupBy("device_id")
.agg(count("session_id").as("session_c"))
.orderBy("device_id")
.persist

/////////////////events all users
var events=df.select("*")
.groupBy("device_id")
.agg(collect_list("event_type").as("all_events"),collect_set("event_type").as("unique_events"))
.withColumn("all_ev",size(col("all_events")))
.withColumn("unique_ev",size(col("unique_events")))
.persist

///bolor segmentner@ hertov joinenq anum heto

In [ ]:
////duration by segment--amenaqich mijin tevoxutyun@ en usenrei mota ov uni session vortex 3ed-a arel?? 
_m3_ed.join(dur,col("id")===col("device_id"),"left").groupBy("c_ed_4").agg(avg("avg_dur")).show()
_3_ed.join(dur,_3_ed.col("id")===dur.col("device_id"),"left").groupBy("c_ed_3").agg(avg("avg_dur")).show()
_1_2_ed.join(dur,col("id")===col("device_id"),"left").groupBy("c_ed_2").agg(avg("avg_dur")).show()
_0_ed.join(dur,col("id")===col("device_id"),"left").groupBy("c_ed_0").agg(avg("avg_dur"))
.show()

In [ ]:
/// freq by segment -- arandznapes ban chi asum
_m3_ed.join(freq,col("id")===col("device_id"),"left").groupBy("c_ed_4").agg(avg("session_c")).show()
_3_ed.join(freq,col("id")===col("device_id"),"left").groupBy("c_ed_3").agg(avg("session_c")).show()
_1_2_ed.join(freq,col("id")===col("device_id"),"left").groupBy("c_ed_2").agg(avg("session_c")).show()
_0_ed.join(freq,col("id")===col("device_id"),"left").groupBy("c_ed_0").agg(avg("session_c")).show()

In [ ]:
//events -- arandznapes ban chi asum
 _m3_ed.join(events,col("id")===col("device_id"),"left").groupBy("c_ed_4").agg(avg("all_ev"),avg("unique_ev"))show()
_3_ed.join(events,col("id")===col("device_id"),"left").groupBy("c_ed_3").agg(avg("all_ev"),avg("unique_ev"))show()
_1_2_ed.join(events,col("id")===col("device_id"),"left").groupBy("c_ed_2").agg(avg("all_ev"),avg("unique_ev"))show()
_0_ed.join(events,col("id")===col("device_id"),"left").groupBy("c_ed_0").agg(avg("all_ev"),avg("unique_ev"))show()